In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from google.colab import files
from google.colab import drive
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import KNNImputer
from sklearn.impute import IterativeImputer
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_path = '/content/drive/My Drive/merged_no_missing.csv'

# Load the CSV data into a DataFrame
data = pd.read_csv(file_path)
data.head(10)

#remove country for later calculation
data = data.drop(columns=["Country"])
data

,T1992,T1993,T1994,T1995,T1996,T1997,T1998,T1999,T2000,T2001,...,D2011,D2012,D2013,D2014,D2015,D2016,D2017,D2018,D2019,D2020
0,-0.294,0.220,0.430,0.359,-0.116,0.471,0.675,1.198,0.993,1.311,...,4.0,10.0,5.0,3.0,5.0,4.0,5.0,5.0,7.0,7.0
1,0.106,0.076,1.330,-0.172,-0.038,0.075,0.795,0.670,1.065,1.532,...,0.0,1.0,0.0,0.0,3.0,1.0,2.0,1.0,0.0,0.0
2,-0.312,0.552,0.732,0.595,0.846,1.059,1.109,1.476,0.820,1.856,...,1.0,2.0,0.0,0.0,2.0,0.0,1.0,1.0,1.0,0.0
3,0.466,0.256,0.212,0.753,0.370,0.107,1.064,0.417,0.169,0.295,...,4.0,1.0,1.0,0.0,3.0,4.0,2.0,1.0,3.0,3.0
4,0.365,0.535,0.575,0.651,0.350,0.459,1.060,0.532,0.264,0.587,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158,0.101,0.332,0.329,0.377,0.001,0.310,1.307,0.740,0.129,0.693,...,5.0,4.0,10.0,3.0,5.0,8.0,9.0,7.0,8.0,11.0
159,-0.061,-0.307,-0.148,0.400,0.336,0.097,0.369,0.238,0.425,0.689,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
160,-1.045,-0.368,0.800,-0.166,0.338,-0.121,0.894,0.900,0.249,0.855,...,0.0,1.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0
161,0.544,-0.117,0.362,0.761,0.393,0.339,0.876,0.064,-0.040,0.263,...,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,2.0,3.0


In [ ]:
miss_data = pd.read_csv('/content/drive/MyDrive/merged_missing.csv')
miss_data_x = miss_data.drop(columns=["Country"])
miss_data_x

,T1992,T1993,T1994,T1995,T1996,T1997,T1998,T1999,T2000,T2001,...,D2011,D2012,D2013,D2014,D2015,D2016,D2017,D2018,D2019,D2020
0,-0.294,0.220,0.430,0.359,-0.116,0.471,0.675,1.198,0.993,1.311,...,4.0,10.0,5.0,3.0,5.0,4.0,5.0,5.0,7.0,7.0
1,0.106,0.076,1.330,-0.172,-0.038,0.075,0.795,0.670,1.065,1.532,...,0.0,1.0,0.0,0.0,3.0,1.0,2.0,1.0,0.0,0.0
2,-0.312,0.552,0.732,0.595,0.846,1.059,1.109,1.476,0.820,1.856,...,1.0,2.0,0.0,0.0,2.0,0.0,1.0,1.0,1.0,0.0
3,0.466,0.256,0.212,0.753,0.370,0.107,1.064,0.417,0.169,0.295,...,4.0,1.0,1.0,0.0,3.0,4.0,2.0,1.0,3.0,3.0
4,0.365,0.535,0.575,0.651,0.350,0.459,1.060,0.532,0.264,0.587,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158,0.101,0.332,0.329,0.377,0.001,0.310,1.307,0.740,0.129,0.693,...,5.0,4.0,10.0,3.0,5.0,8.0,9.0,7.0,8.0,11.0
159,-0.061,-0.307,-0.148,0.400,0.336,0.097,0.369,0.238,0.425,0.689,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
160,-1.045,-0.368,0.800,-0.166,0.338,-0.121,0.894,0.900,0.249,0.855,...,0.0,1.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0
161,0.544,-0.117,0.362,0.761,NaN,0.339,0.876,0.064,-0.040,0.263,...,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,2.0,3.0


# Mean imputation - columnwise

In [ ]:
clms = list(data.columns)
print(clms)

['T1992', 'T1993', 'T1994', 'T1995', 'T1996', 'T1997', 'T1998', 'T1999', 'T2000', 'T2001', 'T2002', 'T2003', 'T2004', 'T2005', 'T2006', 'T2007', 'T2008', 'T2009', 'T2010', 'T2011', 'T2012', 'T2013', 'T2014', 'T2015', 'T2016', 'T2017', 'T2018', 'T2019', 'T2020', 'C1992', 'C1993', 'C1994', 'C1995', 'C1996', 'C1997', 'C1998', 'C1999', 'C2000', 'C2001', 'C2002', 'C2003', 'C2004', 'C2005', 'C2006', 'C2007', 'C2008', 'C2009', 'C2010', 'C2011', 'C2012', 'C2013', 'C2014', 'C2015', 'C2016', 'C2017', 'C2018', 'C2019', 'C2020', 'F1992', 'F1993', 'F1994', 'F1995', 'F1996', 'F1997', 'F1998', 'F1999', 'F2000', 'F2001', 'F2002', 'F2003', 'F2004', 'F2005', 'F2006', 'F2007', 'F2008', 'F2009', 'F2010', 'F2011', 'F2012', 'F2013', 'F2014', 'F2015', 'F2016', 'F2017', 'F2018', 'F2019', 'F2020', 'L1992', 'L1993', 'L1994', 'L1995', 'L1996', 'L1997', 'L1998', 'L1999', 'L2000', 'L2001', 'L2002', 'L2003', 'L2004', 'L2005', 'L2006', 'L2007', 'L2008', 'L2009', 'L2010', 'L2011', 'L2012', 'L2013', 'L2014', 'L2015', 

In [ ]:
#impute the mean value in column direction
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(miss_data_x)
SimpleImputer()
imputed_mean_data_x=imp.transform(miss_data_x)
imputed_mean_data_x #array type, after imputation

#put the titles back, convert it into dataframe
imputed_mean_data_x_df = pd.DataFrame(imputed_mean_data_x, columns=clms)
#print out the dataframe after imputing the
imputed_mean_data_x_df

,T1992,T1993,T1994,T1995,T1996,T1997,T1998,T1999,T2000,T2001,...,D2011,D2012,D2013,D2014,D2015,D2016,D2017,D2018,D2019,D2020
0,-0.294,0.220,0.430,0.359,-0.116000,0.471,0.675,1.198,0.993000,1.311,...,4.0,10.0,5.0,3.0,5.0,4.0,5.0,5.0,7.0,7.0
1,0.106,0.076,1.330,-0.172,-0.038000,0.075,0.795,0.670,1.065000,1.532,...,0.0,1.0,0.0,0.0,3.0,1.0,2.0,1.0,0.0,0.0
2,-0.312,0.552,0.732,0.595,0.846000,1.059,1.109,1.476,0.820000,1.856,...,1.0,2.0,0.0,0.0,2.0,0.0,1.0,1.0,1.0,0.0
3,0.466,0.256,0.212,0.753,0.370000,0.107,1.064,0.417,0.169000,0.295,...,4.0,1.0,1.0,0.0,3.0,4.0,2.0,1.0,3.0,3.0
4,0.365,0.535,0.575,0.651,0.350000,0.459,1.060,0.532,0.264000,0.587,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158,0.101,0.332,0.329,0.377,0.001000,0.310,1.307,0.740,0.129000,0.693,...,5.0,4.0,10.0,3.0,5.0,8.0,9.0,7.0,8.0,11.0
159,-0.061,-0.307,-0.148,0.400,0.336000,0.097,0.369,0.238,0.425000,0.689,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
160,-1.045,-0.368,0.800,-0.166,0.338000,-0.121,0.894,0.900,0.249000,0.855,...,0.0,1.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0
161,0.544,-0.117,0.362,0.761,0.245703,0.339,0.876,0.064,-0.040000,0.263,...,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,2.0,3.0


# Calculate the MSE, RMSE, MAE of the Mean Imputation Method

In [ ]:
# Mask of where values were missing
data=data.to_numpy() #convert data(dataframe) to data(numpy array) for calculation purpose
mask = np.isnan(miss_data_x)
true_values = data[mask]
imputed_values = imputed_mean_data_x[mask]

#calculate the MSE
mse = mean_squared_error(true_values, imputed_values)
print(f"Mean Squred Error (MSE) of mean imputation: {mse}")

#calculate the RMSE
rmse = np.sqrt(mean_squared_error(true_values, imputed_values))
print(f"Root Mean Squred Error (RMSE) of mean imputation: {rmse}")

# Calculate MAE only for the imputed values
mae = mean_absolute_error(true_values, imputed_values)
print(f"Mean Absolute Error (MAE) of mean imputation: {mae}")


Mean Squred Error (MSE) of mean imputation: 0.24101016455725024
Root Mean Squred Error (RMSE) of mean imputation: 0.49092786084846546
Mean Absolute Error (MAE) of mean imputation: 0.3822980058782991


In [ ]:
type(true_values)

numpy.ndarray

# Median imputation - columnwise

In [ ]:
#impute the mean value in column direction
imp = SimpleImputer(missing_values=np.nan, strategy='median')
imp.fit(miss_data_x)
SimpleImputer()
imputed_median_data_x=imp.transform(miss_data_x)
imputed_median_data_x #array type, after imputation

#put the titles back, convert it into dataframe
imputed_median_data_x_df = pd.DataFrame(imputed_median_data_x, columns=clms)
#print out the dataframe after imputing the
imputed_median_data_x_df

,T1992,T1993,T1994,T1995,T1996,T1997,T1998,T1999,T2000,T2001,...,D2011,D2012,D2013,D2014,D2015,D2016,D2017,D2018,D2019,D2020
0,-0.294,0.220,0.430,0.359,-0.116,0.471,0.675,1.198,0.993,1.311,...,4.0,10.0,5.0,3.0,5.0,4.0,5.0,5.0,7.0,7.0
1,0.106,0.076,1.330,-0.172,-0.038,0.075,0.795,0.670,1.065,1.532,...,0.0,1.0,0.0,0.0,3.0,1.0,2.0,1.0,0.0,0.0
2,-0.312,0.552,0.732,0.595,0.846,1.059,1.109,1.476,0.820,1.856,...,1.0,2.0,0.0,0.0,2.0,0.0,1.0,1.0,1.0,0.0
3,0.466,0.256,0.212,0.753,0.370,0.107,1.064,0.417,0.169,0.295,...,4.0,1.0,1.0,0.0,3.0,4.0,2.0,1.0,3.0,3.0
4,0.365,0.535,0.575,0.651,0.350,0.459,1.060,0.532,0.264,0.587,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158,0.101,0.332,0.329,0.377,0.001,0.310,1.307,0.740,0.129,0.693,...,5.0,4.0,10.0,3.0,5.0,8.0,9.0,7.0,8.0,11.0
159,-0.061,-0.307,-0.148,0.400,0.336,0.097,0.369,0.238,0.425,0.689,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
160,-1.045,-0.368,0.800,-0.166,0.338,-0.121,0.894,0.900,0.249,0.855,...,0.0,1.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0
161,0.544,-0.117,0.362,0.761,0.253,0.339,0.876,0.064,-0.040,0.263,...,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,2.0,3.0


# Calculate the MSE, RMSE, MAE of the Median Imputation Method

In [ ]:
# Mask of where values were missing
mask = np.isnan(miss_data_x)
true_values = data[mask]
imputed_values = imputed_median_data_x[mask]

#calculate the MSE
mse = mean_squared_error(true_values, imputed_values)
print(f"Mean Squred Error (MSE) of median imputation: {mse}")

#calculate the RMSE
rmse = np.sqrt(mean_squared_error(true_values, imputed_values))
print(f"Root Mean Squred Error (RMSE) of median imputation: {rmse}")

# Calculate MAE only for the imputed values
mae = mean_absolute_error(true_values, imputed_values)
print(f"Mean Absolute Error (MAE) of median imputation: {mae}")

Mean Squred Error (MSE) of median imputation: 0.24982568502202643
Root Mean Squred Error (RMSE) of median imputation: 0.4998256546257169
Mean Absolute Error (MAE) of median imputation: 0.37655947136563883


# Multivariate feature imputation-Column wise


In [ ]:
imp = IterativeImputer(max_iter=10, random_state=0)
imp.fit(miss_data_x)
IterativeImputer(random_state=0)
multivariate_impute_data_x=imp.transform(miss_data_x)
multivariate_impute_data_x

#put the titles back, convert it into dataframe
multivariate_impute_data_x_df = pd.DataFrame(multivariate_impute_data_x, columns=clms)

#print out the dataframe after imputing the
multivariate_impute_data_x_df

,T1992,T1993,T1994,T1995,T1996,T1997,T1998,T1999,T2000,T2001,...,D2011,D2012,D2013,D2014,D2015,D2016,D2017,D2018,D2019,D2020
0,-0.294,0.220,0.430,0.359,-0.116000,0.471,0.675,1.198,0.99300,1.311,...,4.0,10.0,5.0,3.0,5.0,4.0,5.0,5.0,7.0,7.0
1,0.106,0.076,1.330,-0.172,-0.038000,0.075,0.795,0.670,1.06500,1.532,...,0.0,1.0,0.0,0.0,3.0,1.0,2.0,1.0,0.0,0.0
2,-0.312,0.552,0.732,0.595,0.846000,1.059,1.109,1.476,0.82000,1.856,...,1.0,2.0,0.0,0.0,2.0,0.0,1.0,1.0,1.0,0.0
3,0.466,0.256,0.212,0.753,0.370000,0.107,1.064,0.417,0.16900,0.295,...,4.0,1.0,1.0,0.0,3.0,4.0,2.0,1.0,3.0,3.0
4,0.365,0.535,0.575,0.651,0.350000,0.459,1.060,0.532,0.26400,0.587,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158,0.101,0.332,0.329,0.377,0.001000,0.310,1.307,0.740,0.12900,0.693,...,5.0,4.0,10.0,3.0,5.0,8.0,9.0,7.0,8.0,11.0
159,-0.061,-0.307,-0.148,0.400,0.336000,0.097,0.369,0.238,0.42500,0.689,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
160,-1.045,-0.368,0.800,-0.166,0.338000,-0.121,0.894,0.900,0.24900,0.855,...,0.0,1.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0
161,0.544,-0.117,0.362,0.761,0.242449,0.339,0.876,0.064,-0.04000,0.263,...,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,2.0,3.0


# Calculate the MSE, RMSE, MAE of the multivariate feature Imputation Method

In [ ]:
# Mask of where values were missing
mask = np.isnan(miss_data_x)
true_values = data[mask]
imputed_values = multivariate_impute_data_x[mask]

#calculate the MSE
mse = mean_squared_error(true_values, imputed_values)
print(f"Mean Squred Error (MSE) of multivariate feature imputation: {mse}")

#calculate the RMSE
rmse = np.sqrt(mean_squared_error(true_values, imputed_values))
print(f"Root Mean Squred Error (RMSE) of multivariate feature imputation: {rmse}")

# Calculate MAE only for the imputed values
mae = mean_absolute_error(true_values, imputed_values)
print(f"Mean Absolute Error (MAE) of multivariate feature imputation: {mae}")

Mean Squred Error (MSE) of multivariate feature imputation: 0.14842923593563281
Root Mean Squred Error (RMSE) of multivariate feature imputation: 0.38526515017015595
Mean Absolute Error (MAE) of multivariate feature imputation: 0.27485221534634374


# Nearest neighbors imputation-Column wise

In [ ]:
imputer = KNNImputer(n_neighbors=2, weights="uniform")
KNN_impute_data_x=imputer.fit_transform(miss_data_x)

#put the titles back, convert it into dataframe
KNN_impute_data_x_df = pd.DataFrame(KNN_impute_data_x, columns=clms)

#print out the dataframe after imputing the
KNN_impute_data_x_df

,T1992,T1993,T1994,T1995,T1996,T1997,T1998,T1999,T2000,T2001,...,D2011,D2012,D2013,D2014,D2015,D2016,D2017,D2018,D2019,D2020
0,-0.294,0.220,0.430,0.359,-0.116,0.471,0.675,1.198,0.9930,1.311,...,4.0,10.0,5.0,3.0,5.0,4.0,5.0,5.0,7.0,7.0
1,0.106,0.076,1.330,-0.172,-0.038,0.075,0.795,0.670,1.0650,1.532,...,0.0,1.0,0.0,0.0,3.0,1.0,2.0,1.0,0.0,0.0
2,-0.312,0.552,0.732,0.595,0.846,1.059,1.109,1.476,0.8200,1.856,...,1.0,2.0,0.0,0.0,2.0,0.0,1.0,1.0,1.0,0.0
3,0.466,0.256,0.212,0.753,0.370,0.107,1.064,0.417,0.1690,0.295,...,4.0,1.0,1.0,0.0,3.0,4.0,2.0,1.0,3.0,3.0
4,0.365,0.535,0.575,0.651,0.350,0.459,1.060,0.532,0.2640,0.587,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158,0.101,0.332,0.329,0.377,0.001,0.310,1.307,0.740,0.1290,0.693,...,5.0,4.0,10.0,3.0,5.0,8.0,9.0,7.0,8.0,11.0
159,-0.061,-0.307,-0.148,0.400,0.336,0.097,0.369,0.238,0.4250,0.689,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
160,-1.045,-0.368,0.800,-0.166,0.338,-0.121,0.894,0.900,0.2490,0.855,...,0.0,1.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0
161,0.544,-0.117,0.362,0.761,0.381,0.339,0.876,0.064,-0.0400,0.263,...,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,2.0,3.0


# Calculate the MSE, RMSE, MAE of the KNN Imputation Method

In [ ]:
# Mask of where values were missing
mask = np.isnan(miss_data_x)
true_values = data[mask]
imputed_values = KNN_impute_data_x[mask]

#calculate the MSE
mse = mean_squared_error(true_values, imputed_values)
print(f"Mean Squred Error (MSE) of Nearest neighbors imputation imputation: {mse}")

#calculate the RMSE
rmse = np.sqrt(mean_squared_error(true_values, imputed_values))
print(f"Root Mean Squred Error (RMSE) of Nearest neighbors imputation imputation: {rmse}")

# Calculate MAE only for the imputed values
mae = mean_absolute_error(true_values, imputed_values)
print(f"Mean Absolute Error (MAE) of Nearest neighbors imputation imputation: {mae}")

Mean Squred Error (MSE) of Nearest neighbors imputation imputation: 0.309510308370044
Root Mean Squred Error (RMSE) of Nearest neighbors imputation imputation: 0.556336506414997
Mean Absolute Error (MAE) of Nearest neighbors imputation imputation: 0.41875330396475774
